In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
!pip install openpyxl
!pip install XlsxWriter
!pip install xlrd
!pip install python-dateutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 3.5 MB/s eta 0:00:00


In [3]:
# pd.set_option('display.max_rows', None)

# input_root_path = "../data"
input_root_path = "../input/travel-time"
ats_path = "../input/atstraffic"
weather_path = "../input/weatherstations"
counter_path = "../input/counterstations"
# output_root_path = "../data"
output_root_path = "../working"


In [4]:
def load_csv_to_df(files, filename=None, is_excel=False):
    if is_excel:
        df = pd.concat((pd.read_excel(f) for f in files), ignore_index=True)
    else:
        df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)
    
    if filename:
        df.to_csv(os.path.join(output_root_path, filename), header = True, index=True)
    return df

# Pre-processing

In [5]:
# Load data 

tasks_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive/Travel Time Task/TravelTimeTaskArchive*.csv"))
df_tasks = load_csv_to_df(tasks_joined_files)
# df_tasks = load_csv_to_df(tasks_joined_files, filename="all_TravelTimeTaskArchive.csv")

summary_joined_files = glob.glob(os.path.join(input_root_path, "Update Travel Time Archive/Travel Time Summary/TravelTimeSummaryTaskArchive*.csv"))
# df_summary = load_csv_to_df(summary_joined_files)

df_tasks

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
1,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
2,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
3,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
4,75946582969,Spread 4B - NB,7/8/2021 8:54:49 PM,0.0,0.0,0.0,OK,70.71
...,...,...,...,...,...,...,...,...
13438804,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438805,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438806,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06
13438807,49923813616,MRPP-AP 11 • SB • Ledcor Sicim LP,7/9/2021 12:16:57 AM,0.0,0.0,0.0,OK,3.06


## Find Unique Assets


In [6]:
def get_unique_assets(filename=None):
    df_asset = pd.concat([df_tasks[['AssetNumber', 'AssetIdentifier']], df_summary[['AssetNumber', 'AssetIdentifier']]], axis=0) 
    df_asset_unique = df_asset.drop_duplicates()
    
    if filename:
        df_asset_unique.to_csv(os.path.join(output_root_path, filename), header = True, index=True)

    return df_asset_unique

In [7]:
# df_asset_unique = get_unique_assets()
# df_asset_unique = get_unique_assets(filename='unique_assets.csv')
# df_asset_unique

# Cleaning

## Filter Valid Traffic Times

In [8]:
# Filter out entries with zero UnderConstructionTime or BaselineTime
df_valid = df_tasks[(df_tasks['UnderConstructionTime'] != 0) & (df_tasks['BaselineTime'] != 0)].rename_axis('AllTasksIndex').reset_index()
df_valid = df_valid.rename_axis('ValidTasksIndex')
# df_valid.index.name = 'ValidTasksIndex'
df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
ValidTasksIndex,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,5/23/2021 4:55:09 PM,95.90,95.37,0.53,OK,146.94
1,1000001,59711435013,Spread 7 - NB,5/23/2021 4:55:09 PM,21.63,23.88,0.00,OK,33.60
2,1000002,14058367507,Spread 6 - NB,5/23/2021 4:55:09 PM,40.63,41.88,0.00,OK,72.16
3,1000003,50253718586,Spread 5A - SB,5/23/2021 4:55:10 PM,123.68,115.63,8.05,OK,185.13
4,1000004,20828851697,Spread 5A - NB,5/23/2021 4:55:10 PM,106.97,111.38,0.00,OK,185.21
...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.98
3809837,12938805,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.71
3809838,12938806,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.35


## Get Assets

In [9]:
# df_assets_all_active = pd.read_excel(os.path.join(ats_path, 'AllWorkZones.xlsx'), sheet_name = ['HaveBeenActivated'])['HaveBeenActivated']
# df_assets_all_active

In [10]:
df_assets_recom = pd.read_excel(os.path.join(ats_path, 'Recommended Assets.xlsx'))
df_assets_recom

,Asset Number,Segment Name,Segment Length,Under Construction,Baseline,Direct of Travel,Spread,Traffic Impact,Details of Right of Way to work within or close,Description of work,Road Type,Traffic Control Setup,Easting,Northing,Most Recent Activation,Cell Coverage
0,62134790681,AP 3-B-12-A-1,2.746,1.67,1.72,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
1,10897197432,AP 3-B-12-A-1,2.746,1.68,1.73,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-12-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344630.96633,5.861839e+06,2/25/2022,NaN
2,12768050215,AP 3-B-13-A-1,2.970,1.77,1.92,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
3,56678162306,AP 3-B-13-A-1,2.970,1.80,1.92,SB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of Access Point AP 3-B-13-A-1 along Sp...,Multi-Lane Undivided Roadways,SLAT,344922.86601,5.860906e+06,4/2/2022,NaN
4,86851077697,AP 3-B-13-A-3,18.000,NaN,NaN,NB,Spread 3B,High,I/we request approval to work within the Minis...,For Use of an Access Point along Spread 3-4A o...,Multi-Lane Undivided Roadways,SLAT,346092.86128,5.857159e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,34684428742,T3920.0,4.840,0.00,0.00,SB,Spread 5A,High,SMJV is requesting approval to work within the...,This application is for traffic control at the...,Multi-Lane Divided,Lane Closure,660846.97920,5.552540e+06,NaN,NaN
620,69027783597,T4068.3,3.460,0.00,0.00,SB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
621,92801985490,T4068.3,3.480,0.00,0.00,NB,Spread 5B,High,Freeway posted speed: 120km/h alignment: large...,Lane closure to allow for installation of CRB ...,Multi-Lane Divided,Lane Closure,NaN,NaN,NaN,NaN
622,16764775025,TWS_460042,3.690,2.73,2.58,SB,Reactivation,High,I/W request approval to work within the Minist...,"The eastbound passing lane will be closed, the...",Multi-Lane Undivided Roadways,Lane Closure,356159.86762,5.876918e+06,4/4/2022,No Cell Coverage Area


In [11]:
# # Remove No Cell Coverage Area entries
# df_assets_recom_no_cell = df_assets_recom[df_assets_recom['Cell Coverage'] != 'No Cell Coverage Area']
# df_assets_recom_no_cell

In [12]:
# df_assets_all_active.columns.difference(df_assets_recom.columns)

## Recalculate Delay Time & State

In [13]:
# Recalculate DelayTime 
df_valid['CalculatedDelayTime'] = (df_valid.UnderConstructionTime - df_valid.BaselineTime).clip(lower=0).round(4)
df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime
ValidTasksIndex,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,5/23/2021 4:55:09 PM,95.90,95.37,0.53,OK,146.94,0.53
1,1000001,59711435013,Spread 7 - NB,5/23/2021 4:55:09 PM,21.63,23.88,0.00,OK,33.60,0.00
2,1000002,14058367507,Spread 6 - NB,5/23/2021 4:55:09 PM,40.63,41.88,0.00,OK,72.16,0.00
3,1000003,50253718586,Spread 5A - SB,5/23/2021 4:55:10 PM,123.68,115.63,8.05,OK,185.13,8.05
4,1000004,20828851697,Spread 5A - NB,5/23/2021 4:55:10 PM,106.97,111.38,0.00,OK,185.21,0.00
...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,4/17/2021 6:00:11 PM,21.77,23.60,0.00,OK,32.98,0.00
3809837,12938805,75946582969,Spread 4B - NB,4/17/2021 6:00:12 PM,43.37,45.35,0.00,OK,70.71,0.00
3809838,12938806,43132175891,Spread 5B - SB,4/17/2021 6:00:12 PM,53.48,53.73,0.00,OK,87.35,0.00


In [14]:
# # Recalculate State
# df_valid['CalculatedState'] = np.where(df_valid['CalculatedDelayTime'] < DELAY_THRESH, 'OK', 'DELAY')
# df_valid

## Extract New Columns

### From 'CreatedDate'

In [15]:
# DayType:
#     1: Monday (0) & Friday (4)
#     2: OtherWeekday (1,2,3)
#     3: Weekend (5,6)
dict_day_type = {0:1, 1:2, 2:2, 3:2, 4:1, 5:3, 6:3}

# HourType: 
#     1: 6-10
#     2: 10-15
#     3: 15-18
#     4: 18-22
#     5: 22-6
dict_hour_type = {6:1, 7:1, 8:1,9:1, 
                  10:2,11:2,12:2,13:2,14:2,
                  15:3,16:3,17:3,
                  18:4,19:4,20:4,21:4,
                  22:5,23:5,0:5,1:5,2:5,3:5,4:5,5:5}

In [16]:
df_valid['CreatedDate'] = pd.to_datetime(df_valid['CreatedDate'])
df_valid['CreatedDate:Date'] = df_valid['CreatedDate'].dt.date

# Return the day of the week as an integer, where Monday is 1 and Sunday is 8.
df_valid['CreatedDate:Weekday'] = df_valid['CreatedDate'].dt.weekday + 1
# df_valid['CreatedDate:DayName'] = df_valid['CreatedDate'].dt.day_name()

# df_valid['CreatedDate:WeekOfYear'] = df_valid['CreatedDate'].dt.week_of_year
# df_valid['CreatedDate:DayOfYear'] = df_valid['CreatedDate'].dt.day_of_year

df_valid['CreatedDate:Year'] = df_valid['CreatedDate'].dt.year
df_valid['CreatedDate:Month'] = df_valid['CreatedDate'].dt.month
# # Between 1 and the number of days in the given month of the given year.
df_valid['CreatedDate:Day'] = df_valid['CreatedDate'].dt.day
df_valid['CreatedDate:Hour'] = df_valid['CreatedDate'].dt.hour
# df_valid['CreatedDate:Minute'] = df_valid['CreatedDate'].dt.minute

# Seasons:
# 1: Winter – December, January and February.
# 2: Spring – March, April and May.
# 3: Summer – June, July and August.
# 4: Autumn – September, October and November.
df_valid['CreatedDate:Season'] = df_valid['CreatedDate'].dt.month%12 // 3 + 1

df_valid['CreatedDate:DayType'] = df_valid['CreatedDate'].dt.weekday.map(dict_day_type)

df_valid['CreatedDate:HourType'] = df_valid['CreatedDate'].dt.hour.map(dict_hour_type)

df_valid

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType
ValidTasksIndex,,,,,,,,,,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,2021-05-23 16:55:09,95.90,95.37,0.53,OK,146.94,0.53,2021-05-23,7,2021,5,23,16,2,3,3
1,1000001,59711435013,Spread 7 - NB,2021-05-23 16:55:09,21.63,23.88,0.00,OK,33.60,0.00,2021-05-23,7,2021,5,23,16,2,3,3
2,1000002,14058367507,Spread 6 - NB,2021-05-23 16:55:09,40.63,41.88,0.00,OK,72.16,0.00,2021-05-23,7,2021,5,23,16,2,3,3
3,1000003,50253718586,Spread 5A - SB,2021-05-23 16:55:10,123.68,115.63,8.05,OK,185.13,8.05,2021-05-23,7,2021,5,23,16,2,3,3
4,1000004,20828851697,Spread 5A - NB,2021-05-23 16:55:10,106.97,111.38,0.00,OK,185.21,0.00,2021-05-23,7,2021,5,23,16,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,2021-04-17 18:00:11,21.77,23.60,0.00,OK,32.98,0.00,2021-04-17,6,2021,4,17,18,2,3,4
3809837,12938805,75946582969,Spread 4B - NB,2021-04-17 18:00:12,43.37,45.35,0.00,OK,70.71,0.00,2021-04-17,6,2021,4,17,18,2,3,4
3809838,12938806,43132175891,Spread 5B - SB,2021-04-17 18:00:12,53.48,53.73,0.00,OK,87.35,0.00,2021-04-17,6,2021,4,17,18,2,3,4


### From DelayTime

In [17]:
DELAY_MIN_THRESH = 20
DELAY_MAX_THRESH = 4 * DELAY_MIN_THRESH


In [18]:
# if cond1:
#     exp1
# elif cond2:
#     exp2
# else:
#     exp3
# np.where(cond1, exp1, np.where(cond2, exp2, ...))

df_valid['DelayLevel'] = (
    np.where(df_valid['DelayTime'] == 0.0, 1,
             np.where(df_valid['DelayTime'] > DELAY_MAX_THRESH, 4,
                      np.where(df_valid['DelayTime'] > DELAY_MIN_THRESH, 3, 2))))

print("DELAY_MIN_THRESH =", DELAY_MIN_THRESH)
print("DELAY_MAX_THRESH =", DELAY_MAX_THRESH)

df_valid

DELAY_MIN_THRESH = 20
DELAY_MAX_THRESH = 80


,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
ValidTasksIndex,,,,,,,,,,,,,,,,,,,,
0,1000000,44853071770,Spread Reactivation Hinton to Hargraves - NB,2021-05-23 16:55:09,95.90,95.37,0.53,OK,146.94,0.53,2021-05-23,7,2021,5,23,16,2,3,3,2
1,1000001,59711435013,Spread 7 - NB,2021-05-23 16:55:09,21.63,23.88,0.00,OK,33.60,0.00,2021-05-23,7,2021,5,23,16,2,3,3,1
2,1000002,14058367507,Spread 6 - NB,2021-05-23 16:55:09,40.63,41.88,0.00,OK,72.16,0.00,2021-05-23,7,2021,5,23,16,2,3,3,1
3,1000003,50253718586,Spread 5A - SB,2021-05-23 16:55:10,123.68,115.63,8.05,OK,185.13,8.05,2021-05-23,7,2021,5,23,16,2,3,3,2
4,1000004,20828851697,Spread 5A - NB,2021-05-23 16:55:10,106.97,111.38,0.00,OK,185.21,0.00,2021-05-23,7,2021,5,23,16,2,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809836,12938804,88582155367,Spread 7 - SB,2021-04-17 18:00:11,21.77,23.60,0.00,OK,32.98,0.00,2021-04-17,6,2021,4,17,18,2,3,4,1
3809837,12938805,75946582969,Spread 4B - NB,2021-04-17 18:00:12,43.37,45.35,0.00,OK,70.71,0.00,2021-04-17,6,2021,4,17,18,2,3,4,1
3809838,12938806,43132175891,Spread 5B - SB,2021-04-17 18:00:12,53.48,53.73,0.00,OK,87.35,0.00,2021-04-17,6,2021,4,17,18,2,3,4,1


In [19]:
# df_valid.loc[df_valid['DelayLevel'] == 3]    

In [20]:
# df_valid.to_csv(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.csv'), header = True, index=True)
# df_valid.to_excel(os.path.join(output_root_path, 'AllValidEdited_TravelTimeTaskArchive.xlsx'), header = True, index=True)

## Match work zone datasets
For all the workzones under the "have been activated" tab, creates a separate spreadsheet for each workzone that contains all the travel time information that you can find in the "Update Travel Time Archive_All_Data" spreadsheet. So for each workzone we would have one spreadsheet with all the travel time information.

In [21]:
def find_workzone_tt_data(df_assets, file_name=None):
    df_tt_workzone = pd.DataFrame()
    
    for i, location_id in enumerate(df_assets['Asset Number']):
        df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
        df_tt_workzone = df_tt_workzone.append(df_intersect)
        
        if file_name:
            with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
                df_intersect.to_excel(writer, sheet_name=str(location_id))
            
        if (i+1) % 100 == 0:
            print(f'{round((i+1)/len(df_assets)*100, 2)} %')
            
    return df_tt_workzone
         

In [22]:
df_tt_workzone = find_workzone_tt_data(df_assets_recom)
df_tt_workzone

16.03 %
32.05 %
48.08 %
64.1 %
80.13 %
96.15 %


,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel
ValidTasksIndex,,,,,,,,,,,,,,,,,,,,
87153,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,2021-05-30,7,2021,5,30,20,2,3,4,2
87203,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2
87253,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2
87303,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1
87353,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452149,12552139,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:40:11,1.00,1.65,0.00,OK,2.73,0.00,2021-03-18,4,2021,3,18,3,2,2,5,1
3452228,12552231,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:45:11,1.00,1.65,0.00,OK,2.73,0.00,2021-03-18,4,2021,3,18,3,2,2,5,1
3452301,12552314,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:50:12,1.00,1.65,0.00,OK,2.73,0.00,2021-03-18,4,2021,3,18,3,2,2,5,1


In [23]:
# find_data_workzone(df_assets_all_active, file_name='TravelTimesPerWorkZone_AllActive.xlsx')
# find_data_workzone(df_assets_recom, file_name=''TravelTimesPerWorkZone_Recommended.xlsx')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_AllActive.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_all_active['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_all_active)*100, 2)} %')

# # with pd.ExcelWriter('TravelTimesPerWorkZone_Recommended.xlsx', engine='xlsxwriter') as writer:
# #     for i, location_id in enumerate(df_assets_recom['Asset Number']):
# #         df_intersect = df_valid[df_valid['AssetNumber'] == location_id]
# #         df_intersect.to_excel(writer, sheet_name=str(location_id))
# #         if (i+1) % 20 == 0:
# #             print(f'{round((i+1)/len(df_assets_recom)*100, 2)} %')


# Get Weather Data

In [24]:
df_matched_ids = pd.read_csv(os.path.join(ats_path, 'AssetNumberStationID.csv'), index_col=0)
df_matched_ids.rename(columns = {'Asset Number': 'AssetNumber', 'Station ID': 'Weather:StationID'}, inplace=True)
df_matched_ids

,AssetNumber,Weather:StationID
0,23023696729,706
1,98128214373,706
2,73174024548,706
3,40105790845,706
4,12738620179,706
...,...,...
619,55858030442,794
620,45364364608,794
621,40786939101,794
622,34684428742,49408


In [25]:
df_weather = pd.DataFrame()

# for station_id in df_matched_ids['Weather:StationID'].unique():
for folder_name in os.listdir(weather_path):
    station_id = int(folder_name)
    weather_joined_files = glob.glob(os.path.join(weather_path, f'{station_id}/*.csv'))
    df_weather_station = load_csv_to_df(weather_joined_files)
    
    df_weather_station.insert(loc=0, column='StationID', value=station_id)
    df_weather_station['Date/Time (LST)'] = pd.to_datetime(df_weather_station['Date/Time (LST)'])
    # df_weather_station['Date'] = df_weather_station['Date/Time (LST)'].dt.date
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Date/Time (LST)')+1, column='Date', value= df_weather_station['Date/Time (LST)'].dt.date)
    df_weather_station.insert(loc=df_weather_station.columns.get_loc('Time (LST)')+1, column='Hour', value= df_weather_station['Date/Time (LST)'].dt.hour)

    df_weather = pd.concat([df_weather, df_weather_station[['StationID', 'Longitude (x)', 'Latitude (y)', 'Date', 'Hour', 'Temp (°C)', 'Precip. Amount (mm)', 'Wind Spd (km/h)']]], axis=0)
    
df_weather.rename(columns = {'Temp (°C)':'Temp',
                             'Precip. Amount (mm)':'Precip',
                             'Wind Spd (km/h)':'WindSpd'},
                  inplace=True)
df_weather.columns = ['Weather:' + str(col) for col in df_weather.columns]

df_weather

,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,707,-121.76,49.24,2022-07-01,0,NaN,NaN,NaN
1,707,-121.76,49.24,2022-07-01,1,NaN,NaN,NaN
2,707,-121.76,49.24,2022-07-01,2,NaN,NaN,NaN
3,707,-121.76,49.24,2022-07-01,3,NaN,NaN,NaN
4,707,-121.76,49.24,2022-07-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26299,8214,-119.29,52.13,2020-01-31,19,0.1,1.0,3.0
26300,8214,-119.29,52.13,2020-01-31,20,0.1,0.0,2.0
26301,8214,-119.29,52.13,2020-01-31,21,0.1,0.3,3.0
26302,8214,-119.29,52.13,2020-01-31,22,0.3,0.0,3.0


## Add Weather Data to Traffic Time Data

In [26]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_ids, left_on='AssetNumber', right_on='AssetNumber')
df_tt_workzone

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,...,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID
0,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,...,7,2021,5,30,20,2,3,4,2,1355
1,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,...,7,2021,5,30,20,2,3,4,2,1355
2,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,...,7,2021,5,30,20,2,3,4,2,1355
3,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,...,7,2021,5,30,20,2,3,4,1,1355
4,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,...,7,2021,5,30,20,2,3,4,1,1355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367734,12552139,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:40:11,1.00,1.65,0.00,OK,2.73,0.00,...,4,2021,3,18,3,2,2,5,1,10223
1367735,12552231,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:45:11,1.00,1.65,0.00,OK,2.73,0.00,...,4,2021,3,18,3,2,2,5,1,10223
1367736,12552314,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:50:12,1.00,1.65,0.00,OK,2.73,0.00,...,4,2021,3,18,3,2,2,5,1,10223
1367737,12552361,42450766624,MRPP-AP 9-B • NB • Ledcor Sicim LP • 419.2,2021-03-18 03:55:11,1.00,1.65,0.00,OK,2.73,0.00,...,4,2021,3,18,3,2,2,5,1,10223


In [27]:
df_tt_workzone = pd.merge(left = df_tt_workzone, right = df_weather,
                          left_on = ['Weather:StationID', 'CreatedDate:Date', 'CreatedDate:Hour'],
                          right_on = ['Weather:StationID', 'Weather:Date', 'Weather:Hour'])
df_tt_workzone = df_tt_workzone.drop(columns=['Weather:Date', 'Weather:Hour'])

df_tt_workzone

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,...,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd
0,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,...,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN
1,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,...,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN
2,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,...,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN
3,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,...,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN
4,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,...,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367734,11283448,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:35:11,1.92,1.93,0.00,OK,3.24,0.00,...,2,2,4,1,10223,-118.03,52.93,17.5,0.0,14.0
1367735,11283492,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:40:16,1.92,1.93,0.00,OK,3.24,0.00,...,2,2,4,1,10223,-118.03,52.93,17.5,0.0,14.0
1367736,11283535,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:45:12,1.92,1.93,0.00,OK,3.24,0.00,...,2,2,4,1,10223,-118.03,52.93,17.5,0.0,14.0
1367737,11283571,42943539227,MRPP-AP 6 • NB • Ledcor Sicim LP • 414,2021-05-26 18:50:13,1.92,1.93,0.00,OK,3.24,0.00,...,2,2,4,1,10223,-118.03,52.93,17.5,0.0,14.0


### Checking Weather data

In [28]:
 df_weather[(df_weather['Weather:Date'] == df_tt_workzone.iloc[0]['CreatedDate:Date']) &
            (df_weather['Weather:Hour'] == 20) & 
            (df_weather['Weather:StationID'] == 1355)]
    
# df_tt_workzone.iloc[0]['CreatedDate:Date']

,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
23348,1355,-119.47,52.72,2021-05-30,20,NaN,NaN,NaN


In [29]:
df_matched_ids[df_matched_ids['Weather:StationID'] == 1355] 

,AssetNumber,Weather:StationID
84,71808680042,1355
85,51881464571,1355
86,53330524217,1355
87,61325418624,1355
88,50744787359,1355
...,...,...
167,30783661221,1355
168,92281098181,1355
169,10897197432,1355
170,64277836054,1355


In [30]:
df_weather[df_weather['Weather:Temp'].isnull()]

,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Date,Weather:Hour,Weather:Temp,Weather:Precip,Weather:WindSpd
0,707,-121.76,49.24,2022-07-01,0,NaN,NaN,NaN
1,707,-121.76,49.24,2022-07-01,1,NaN,NaN,NaN
2,707,-121.76,49.24,2022-07-01,2,NaN,NaN,NaN
3,707,-121.76,49.24,2022-07-01,3,NaN,NaN,NaN
4,707,-121.76,49.24,2022-07-01,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
23421,8214,-119.29,52.13,2022-10-31,21,NaN,NaN,NaN
23422,8214,-119.29,52.13,2022-10-31,22,NaN,NaN,NaN
23423,8214,-119.29,52.13,2022-10-31,23,NaN,NaN,NaN
26247,8214,-119.29,52.13,2020-01-29,15,NaN,NaN,NaN


In [31]:
# def find_weather_info(tt_entry, dict_tt_help_idx):
#     station_id = dict_matched_ids[tt_entry[dict_tt_help_idx['AssetNumber']]] 
#     weather_data = dict_weather[station_id]

#     weather_record = weather_data[(weather_data['Date/Time (LST)'].dt.date == tt_entry[dict_tt_help_idx['CreatedDate']].date())
#                                   & (weather_data['Date/Time (LST)'].dt.hour == tt_entry[dict_tt_help_idx['CreatedDate']].hour)]

#     return weather_record


# def dict_funct():
#     temp = df_tt_workzone.to_dict('records')
#     tt_idx = 0

#     for tt_entry in temp:
#         if tt_entry['AssetNumber'] in dict_matched_ids:
#             weather_record = find_weather_info(tt_entry)

#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Temp']] = weather_record['Temp (°C)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:Precip']] = weather_record['Precip. Amount (mm)'].values[0]
#             df_tt_workzone.iat[tt_idx, dict_tt_help_idx['Weather:WindSpd']] = weather_record['Wind Spd (km/h)'].values[0]

#         if tt_idx % 10000 == 0:
#             print(f'{(tt_idx+1)/len(df_tt_workzone)*100:.2f}%')
#         tt_idx += 1

#     return df_tt_workzone


# def add_weather_data(df_tt, start, stop):
#     print(start, stop)
#     df_tt = df_tt.reset_index()

#     df_tt['Weather:Temp'] = ''
#     df_tt['Weather:Precip'] = ''
#     df_tt['Weather:WindSpd'] = ''

#     dict_tt_help_idx = {'Weather:Temp': df_tt.columns.get_loc('Weather:Temp'),
#                         'Weather:Precip': df_tt.columns.get_loc('Weather:Precip'),
#                         'Weather:WindSpd': df_tt.columns.get_loc('Weather:WindSpd'),
#                         'CreatedDate': df_tt.columns.get_loc('CreatedDate'),
#                         'AssetNumber': df_tt.columns.get_loc('AssetNumber')}
    
#     df_tt_workzone = dict_funct(df_tt, dict_tt_help_idx, start, stop)
#     df_tt_workzone.to_csv(os.path.join(output_root_path, f'processed_travel_time_{start}-{stop}.csv'))  
#     return df_tt_workzone

# Get Counters Data


In [32]:
df_matched_counters_ids = pd.read_excel(os.path.join(counter_path, 'NearestCountStationsToAssets.xlsx'))[['Asset_Numb', 'prefixSiteNo']]
df_matched_counters_ids.rename(columns = {'Asset_Numb': 'AssetNumber', 'prefixSiteNo': 'Traffic:SiteNo'}, inplace=True)
df_matched_counters_ids

,AssetNumber,Traffic:SiteNo
0,23023696729,17-044EW
1,98128214373,17-044EW
2,25033704574,23-009NS
3,71808680042,23-010NS
4,27743462715,23-010NS
...,...,...
619,12066281555,21-011NS
620,62134790681,23-010NS
621,33379829945,23-010NS
622,21272146931,P-27-1EW


In [33]:
def clean_traffic_data(df):
    if df.shape[1] == 35:
        df = df.drop(columns=['Unnamed: 7', 'Unnamed: 31', 'Unnamed: 34'])
    elif df.shape[1] == 34: 
        df = df.drop(columns=[ 'Unnamed: 30', 'Unnamed: 33'])
    elif df.shape[1] == 32:
        df = df.drop(columns=['Unnamed: 28', 'Unnamed: 31'])
    elif df.shape[1] == 30:
        df = df.drop(columns=['Unnamed: 26', 'Unnamed: 29'])
    
    df = df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 9', 'Unnamed: 17'])
    df = df.dropna(how='all', axis=1)
    df = df.dropna(axis=0)
    
    df.rename(columns = {'Unnamed: 6':'Date'}, inplace = True)
    df = df.reset_index(drop=True)
    return df

In [34]:
ignore_stations = ['21-012NS', '23-005EW', 'W-23-2EW']

In [35]:
df_traffic = {'Temp': pd.DataFrame(), 'Perm': pd.DataFrame()}

# for site_no in df_matched_counters_ids['prefixSiteNo'].unique():
for site_no in os.listdir(os.path.join(counter_path, f'TrafficInfo')):
    if site_no in ignore_stations:
        continue
        
    site_type = 'DV01' if site_no.startswith('P') else 'DV03S' 
    
    counter_joined_files = glob.glob(os.path.join(counter_path, f'TrafficInfo/{site_no}/{site_type}*.xls'))
    for f in counter_joined_files:
        raw_traffic_site = pd.read_excel(f, header=5, thousands=',')
    
    df_traffic_site = pd.DataFrame()
#     df_traffic_site['Year'] = int(counter_joined_files[0][-8:-4])
# #     df_traffic_site['CreatedDate'].year - 1

    if site_no.startswith('P'):
#         print("\n****BINGO***")
#         print(site_no, raw_traffic_site.shape, '\n')
        clean_traffic_site = clean_traffic_data(raw_traffic_site)
        temp_datetime = pd.to_datetime(clean_traffic_site['Date'])
        df_traffic_site['BaseYear'] = temp_datetime.dt.year
        df_traffic_site['Month'] = temp_datetime.dt.month
        df_traffic_site['Day'] = temp_datetime.dt.day
        df_traffic_site['AADT'] = clean_traffic_site['Road'].astype(float)
        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=False)
        df_traffic_site.insert(loc=len(df_traffic_site.columns), column='CreatedDate', value=pd.to_datetime(raw_traffic_site.iloc[-1][3]))
        df_traffic['Perm'] = pd.concat([df_traffic['Perm'], df_traffic_site], axis=0)
    else:
        temp_datetime = pd.to_datetime(counter_joined_files[0][-8:-4]).date()   # YEAR in datetime format    
        df_traffic_site['BaseYear'] = [temp_datetime.year]
        df_traffic_site['Month'] = [temp_datetime.month]
        df_traffic_site['Day'] = [temp_datetime.day]
        df_traffic_site['AADT'] = [float(raw_traffic_site.iloc[-1][13])]
        df_traffic_site.insert(loc=0, column='SiteNo', value=site_no)
        df_traffic_site.insert(loc=1, column='IsTemporary', value=True)
        df_traffic_site['CreatedDate'] = [pd.to_datetime(raw_traffic_site.iloc[-1][1]).date()]
        df_traffic['Temp'] = pd.concat([df_traffic['Temp'], df_traffic_site], axis=0)
#         break
        
for site_type in df_traffic:
    df_traffic[site_type].columns = ['Traffic:' + str(col) for col in df_traffic[site_type].columns]
    df_traffic[site_type] = df_traffic[site_type].reset_index(drop=True)

df_traffic['Perm']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
0,P-17-1EW,False,2021,1,1,9251.0,2022-03-08
1,P-17-1EW,False,2021,1,2,12042.0,2022-03-08
2,P-17-1EW,False,2021,1,3,13890.0,2022-03-08
3,P-17-1EW,False,2021,1,4,10507.0,2022-03-08
4,P-17-1EW,False,2021,1,5,10812.0,2022-03-08
...,...,...,...,...,...,...,...
1825,P-23-3NS,False,2021,12,27,1668.0,2022-03-09
1826,P-23-3NS,False,2021,12,28,1751.0,2022-03-09
1827,P-23-3NS,False,2021,12,29,1748.0,2022-03-09
1828,P-23-3NS,False,2021,12,30,1751.0,2022-03-09


In [36]:
df_traffic['Temp']

,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
0,27-001E,True,2013,1,1,9487.004,2014-01-20
1,21-937NS,True,2006,1,1,3767.000,2007-02-14
2,27-022E,True,2013,1,1,7449.458,2014-01-20
3,27-0404,True,2013,1,1,3232.165,2014-01-20
4,27-001W,True,2013,1,1,9435.631,2014-01-20
5,21-011NS,True,2017,1,1,1352.857,2018-01-25
6,21-013EW,True,2017,1,1,402.704,2018-01-25
7,23-001NS,True,2017,1,1,1542.281,2018-01-25
8,27-003EW,True,2013,1,1,3608.106,2014-01-20
9,21-006NS,True,2016,1,1,3140.103,2017-02-09


## Add Counters Data to Traffic Time Data

In [37]:
pd.set_option('display.max_columns', None)

In [38]:
GROWTH_RATE = 0.01

In [39]:
df_tt_workzone = pd.merge(left=df_tt_workzone, right=df_matched_counters_ids, left_on='AssetNumber', right_on='AssetNumber')
df_tt_workzone

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo
0,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
1,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
2,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
3,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
4,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367734,11887895,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:15:10,1.67,1.67,0.00,OK,2.743,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS
1367735,12934069,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:00:11,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS
1367736,12934113,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:05:10,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS
1367737,12934157,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:10:11,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS


In [40]:
def find_diff(a, b):
    return list(set(a).difference(b))
    

# set(df_tt_workzone['Traffic:SiteNo'].unique()).difference(set(os.listdir(os.path.join(counter_path, f'TrafficInfo'))))
missing_site = find_diff(df_tt_workzone['Traffic:SiteNo'].unique(), os.listdir(os.path.join(counter_path, f'TrafficInfo')))

missing_data_count = len(df_tt_workzone[df_tt_workzone['Traffic:SiteNo'].isin(ignore_stations + missing_site)])
missing_data_count

44815

In [41]:
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='left')
# tt = pd.merge(left = df_tt_workzone, right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')
join_temp_traffic = df_tt_workzone.reset_index().merge(
    df_traffic['Temp'], on = 'Traffic:SiteNo', how="inner").set_index('index')
join_temp_traffic

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1087588,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.75,0.03,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
1,1087639,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
2,1087690,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.75,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
3,1087741,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
4,1087792,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.75,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017,1,1,1900.944,2018-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154436,10943695,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20
1154437,10943728,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20
1154438,10943761,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,1,1,10918.750,2014-01-20


In [42]:
# tp = pd.merge(left = df_tt_workzone, right = df_traffic['Perm'], 
#          left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
#          right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day'],
#              how='inner')
join_perm_traffic = df_tt_workzone.reset_index().merge(df_traffic['Perm'], how="inner",
                                                       left_on = ['Traffic:SiteNo', 'CreatedDate:Month', 'CreatedDate:Day'],
                                                       right_on = ['Traffic:SiteNo', 'Traffic:Month', 'Traffic:Day']
                                                      ).set_index('index')
join_perm_traffic

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:Month,Traffic:Day,Traffic:AADT,Traffic:CreatedDate
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94623,1087575,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:25:12,3.10,3.0,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
94624,1087626,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:30:13,3.10,3.0,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
94625,1087677,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:35:18,3.07,3.0,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
94626,1087728,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:40:16,3.07,3.0,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
94627,1087779,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:45:14,3.07,3.0,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,5,30,2393.0,2022-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713446,2422323,44374480886,MRPP-AP 37 • SB • Ledcor Sicim LP • 462.6,2021-04-22 05:05:11,1.18,1.2,0.00,OK,1.81,0.00,2021-04-22,4,2021,4,22,5,2,2,5,1,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2021,4,22,3409.0,2022-03-09
713447,2422369,44374480886,MRPP-AP 37 • SB • Ledcor Sicim LP • 462.6,2021-04-22 05:10:11,1.17,1.2,0.00,OK,1.81,0.00,2021-04-22,4,2021,4,22,5,2,2,5,1,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2021,4,22,3409.0,2022-03-09
713448,10806495,44374480886,MRPP-AP 37 • SB • Ledcor Sicim LP • 462.6,2021-04-22 05:00:12,1.18,1.2,0.00,OK,1.81,0.00,2021-04-22,4,2021,4,22,5,2,2,5,1,1355,-119.47,52.72,NaN,NaN,NaN,P-23-2EW,False,2021,4,22,3409.0,2022-03-09


In [43]:
df_tt_workzone_join = pd.concat([join_perm_traffic, join_temp_traffic], axis=0)
df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join = df_tt_workzone_join.drop(columns=['Traffic:Month', 'Traffic:Day', 'Traffic:CreatedDate'])
df_tt_workzone_join['Traffic:CalculatedAADT'] = df_tt_workzone_join['Traffic:AADT'] * (1 + (GROWTH_RATE * (df_tt_workzone_join['CreatedDate:Year'] - df_tt_workzone_join['Traffic:BaseYear'])))
df_tt_workzone_join

,AllTasksIndex,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94623,1087575,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:25:12,3.10,3.00,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
94624,1087626,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:30:13,3.10,3.00,0.10,OK,5.23,0.10,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
94625,1087677,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:35:18,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
94626,1087728,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:40:16,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
94627,1087779,50195678222,"AP 3-A-4, AP 3-B-5, AP 3-B-6 • SB • Ledcor Sic...",2021-05-30 20:45:14,3.07,3.00,0.07,OK,5.23,0.07,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,P-23-3NS,False,2021,2393.00,2393.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154436,10943695,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25
1154437,10943728,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25
1154438,10943761,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,0.00,2021-05-01,6,2021,5,1,18,2,3,4,1,706,-121.76,49.24,14.2,0.0,6.0,17-015W,True,2013,10918.75,11792.25


In [44]:
incomplete_idx = find_diff(df_tt_workzone.index.values, df_tt_workzone_join.index.values)
incomplete_entries = df_tt_workzone[df_tt_workzone.index.isin(incomplete_idx)]
print('# incomplete_entries:', len(incomplete_entries))

df_tt_workzone = pd.concat([df_tt_workzone_join, incomplete_entries], axis=0).sort_index()
df_tt_workzone = df_tt_workzone.drop(columns='AllTasksIndex')
df_tt_workzone

# incomplete_entries: 55698


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,CalculatedDelayTime,CreatedDate:Date,CreatedDate:Weekday,CreatedDate:Year,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:Season,CreatedDate:DayType,CreatedDate:HourType,DelayLevel,Weather:StationID,Weather:Longitude (x),Weather:Latitude (y),Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:BaseYear,Traffic:AADT,Traffic:CalculatedAADT
0,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:25:14,1.73,1.70,0.03,OK,2.750,0.03,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
1,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:30:15,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
2,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:35:20,1.72,1.70,0.02,OK,2.750,0.02,2021-05-30,7,2021,5,30,20,2,3,4,2,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
3,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:40:18,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
4,62134790681,AP 3-B-12-A-1 • NB • Ledcor Sicim LP • 511.9,2021-05-30 20:45:15,1.70,1.70,0.00,OK,2.750,0.00,2021-05-30,7,2021,5,30,20,2,3,4,1,1355,-119.47,52.72,NaN,NaN,NaN,23-010NS,True,2017.0,1900.944,1976.98176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367734,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:15:10,1.67,1.67,0.00,OK,2.743,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS,True,2017.0,1900.944,1976.98176
1367735,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:00:11,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS,True,2017.0,1900.944,1976.98176
1367736,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:05:10,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS,True,2017.0,1900.944,1976.98176
1367737,32553085010,MRPP-AP 5 • SB • Ledcor Sicim LP • 413.1,2021-04-17 06:10:11,1.67,1.67,0.00,OK,2.740,0.00,2021-04-17,6,2021,4,17,6,2,3,1,1,10223,-118.03,52.93,-6.2,0.0,4.0,23-010NS,True,2017.0,1900.944,1976.98176


In [45]:
df_tt_workzone.columns

Index(['AssetNumber', 'AssetIdentifier', 'CreatedDate',
       'UnderConstructionTime', 'BaselineTime', 'DelayTime', 'State',
       'SegmentLength', 'CalculatedDelayTime', 'CreatedDate:Date',
       'CreatedDate:Weekday', 'CreatedDate:Year', 'CreatedDate:Month',
       'CreatedDate:Day', 'CreatedDate:Hour', 'CreatedDate:Season',
       'CreatedDate:DayType', 'CreatedDate:HourType', 'DelayLevel',
       'Weather:StationID', 'Weather:Longitude (x)', 'Weather:Latitude (y)',
       'Weather:Temp', 'Weather:Precip', 'Weather:WindSpd', 'Traffic:SiteNo',
       'Traffic:IsTemporary', 'Traffic:BaseYear', 'Traffic:AADT',
       'Traffic:CalculatedAADT'],
      dtype='object')

In [46]:
# pd.merge(left = tp[:count], right = tt[:count], on = ['Traffic:SiteNo','Traffic:IsTemporary'], how='inner')
# pd.merge(left = tp[:count], right = df_traffic['Temp'], on = 'Traffic:SiteNo', how='inner')

In [47]:
df_tt_workzone.to_csv(os.path.join(output_root_path, 'TravelTime_Valid_Recom_Weather_Traffic.csv'))

In [48]:
df_tt_workzone.to_csv(os.path.join(output_root_path, 'HEADERTravelTime_Valid_Recom_Weather_Traffic.csv'), header = True, index=True)

# Add Speed Limit Data

## Export Potential Wrong Data

In [49]:
def find_delay_diff():
    diff_delay = df_valid[df_valid['DelayTime'] != df_valid['CalculatedDelayTime']]
    diff_delay.to_csv(os.path.join(output_root_path, 'wrong_DelayTime.csv'), header = True, index=True)
    return diff_delay

In [50]:
def find_state_diff():
    diff_state = df_valid[df_valid['State'] != df_valid['CalculatedState']]
    diff_state.to_csv(os.path.join(output_root_path, 'wrong_State.csv'), header = True, index=True)
    return diff_state 

# Exploratory Data Analysis

## General Info about dataframe

In [51]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3809841 entries, 0 to 3809840
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   AllTasksIndex          int64         
 1   AssetNumber            int64         
 2   AssetIdentifier        object        
 3   CreatedDate            datetime64[ns]
 4   UnderConstructionTime  float64       
 5   BaselineTime           float64       
 6   DelayTime              float64       
 7   State                  object        
 8   SegmentLength          float64       
 9   CalculatedDelayTime    float64       
 10  CreatedDate:Date       object        
 11  CreatedDate:Weekday    int64         
 12  CreatedDate:Year       int64         
 13  CreatedDate:Month      int64         
 14  CreatedDate:Day        int64         
 15  CreatedDate:Hour       int64         
 16  CreatedDate:Season     int64         
 17  CreatedDate:DayType    int64         
 18  CreatedDate:HourType  

In [52]:
df_valid.isnull().sum()

AllTasksIndex              0
AssetNumber                0
AssetIdentifier            0
CreatedDate                0
UnderConstructionTime      0
BaselineTime               0
DelayTime                  0
State                    178
SegmentLength              0
CalculatedDelayTime        0
CreatedDate:Date           0
CreatedDate:Weekday        0
CreatedDate:Year           0
CreatedDate:Month          0
CreatedDate:Day            0
CreatedDate:Hour           0
CreatedDate:Season         0
CreatedDate:DayType        0
CreatedDate:HourType       0
DelayLevel                 0
dtype: int64

In [53]:
print(max(df_valid['CreatedDate:Month']))
print(min(df_valid['CreatedDate:Month']))

12
1


In [54]:
print(max(df_valid['CreatedDate:Year']))
print(min(df_valid['CreatedDate:Year']))

2021
2020


## Plot

In [55]:
import seaborn as sns
import matplotlib.pyplot as plt
# from PIL import Image

%matplotlib inline
sns.set_style("darkgrid")
plt.style.use('ggplot')

STEP_SIZE = 0.5


In [56]:
def draw_time_plot(loc, loc_id, time_type, plt_name):
    fig, ax = plt.subplots()

    # loc.plot.scatter(x='CreatedDate:DayName', y=['UnderConstructionTime', 'BaselineTime'])
    # loc.plot.scatter(x='CreatedDate:DayName', y='UnderConstructionTime')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['UnderConstructionTime'], color='purple', marker='o', label='UnderConstructionTime (After)')
#     plt.scatter(x=loc[f'CreatedDate:{time_type}'], y=loc['BaselineTime'], color='orange', marker='h', label='BaselineTime (Before)')

    x, y  = [], []
    
    if time_type == 'Hour':
        for hour in range(24):
            loc_hour = loc.loc[loc[f'CreatedDate:{time_type}'] == hour]
            x.append(hour)
            y.append(loc_hour['UnderConstructionTime'].mean())
    
    elif time_type == 'DayName':
         for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            loc_day = loc.loc[loc[f'CreatedDate:{time_type}'] == day]
            x.append(day)
            y.append(loc_day['UnderConstructionTime'].mean())
    
    temp = loc['BaselineTime'].mean()
    plt.plot(x, y, color='purple', marker='o', label='UnderConstructionTime (After)')
    plt.plot(x, [temp for i in range(len(y))], color='orange', marker='h', label='BaselineTime (Before)')
    
#     start, end = ax.get_ylim()
#     plt.yticks(np.arange(np.round(start), np.round(end)+STEP_SIZE, step=STEP_SIZE))
    if time_type == 'Hour':
        plt.xticks(np.arange(0, 24, step=1))
    
    fig.set_size_inches(14, 10)
    ax.legend()
    ax.grid(True)
    plt.title(f'Travel Time for AssetID {loc_id} ({time_type})')
    plt.xlabel(time_type)
    plt.ylabel('TravelTime')
    plt.tight_layout()
    plt.savefig(f'{output_root_path}/{plt_name}.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


### Draw plot for selected asset ids

In [57]:
locations_id = [80002045894, 50253718586, 44853071770, 12738620179, 37528363723, 43805438913, 68881926366, 36674989230, 34592545857, 22685608926]


In [58]:
# for location_id in locations_id:
#     for time_type in ['DayName', 'Hour']:
#         location = df_valid.loc[df_valid['AssetNumber'] == location_id]
#         draw_time_plot(location, location_id, time_type, plt_name=f'{location_id}_{time_type}')


In [59]:
# !zip plots.zip ../working/*.png